In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../input/train_folds.csv")
df.head(3)

,id,target,file_path,fold
0,00000e74ad,1,input/train/0/0/0/00000e74ad.npy,2
1,00001f4945,0,input/train/0/0/0/00001f4945.npy,1
2,0000661522,0,input/train/0/0/0/0000661522.npy,3


In [3]:
import torch

In [5]:
n_folds = 5

In [6]:
states_b0 = [torch.load(f"../final_models/v_5/b0/tf_efficientnet_b0_ns_fold{fold}_best_score.pth") for fold in range(n_folds)]

In [7]:
states_b1 = [torch.load(f"../final_models/v_5/b1/tf_efficientnet_b1_ns_fold{fold}_best_score.pth") for fold in range(n_folds)]

In [8]:
# states_b2 = [torch.load(f"../final_models/v_4/b2_3d_v4/tf_efficientnet_b2_ns_fold{fold}_best_score.pth") for fold in range(n_folds)]

In [9]:
# states_b7 = [torch.load(f"../final_models/v_4/b7_3d_v4/tf_efficientnet_b7_ns_fold{fold}_best_score.pth") for fold in range(n_folds)]

In [10]:
oof_df = pd.DataFrame()
for fold in range(n_folds):
    _df = df[df['fold'] == fold].reset_index(drop=True)
    preds_b0 = states_b0[fold]['preds'].flatten()
    preds_b1 = states_b1[fold]['preds'].flatten()
    # preds_b2 = states_b2[fold]['preds'].flatten()
    # preds_b7 = states_b7[fold]['preds'].flatten()
    
    _df['preds_b0'] = preds_b0
    _df['preds_b1'] = preds_b1
    # _df['preds_b2'] = preds_b2
    # _df['preds_b7'] = preds_b7
    oof_df = pd.concat([oof_df, _df])

In [11]:
oof_df.head(10)

,id,target,file_path,fold,preds_b0,preds_b1
0,00010beb4a,1,input/train/0/0/0/00010beb4a.npy,0,0.448730,0.375732
1,00014b7a9d,1,input/train/0/0/0/00014b7a9d.npy,0,0.214233,0.172852
2,0001c76f56,1,input/train/0/0/0/0001c76f56.npy,0,0.676270,0.571777
3,0002b883f5,0,input/train/0/0/0/0002b883f5.npy,0,0.173096,0.201660
4,00034d8429,0,input/train/0/0/0/00034d8429.npy,0,0.187988,0.167114
5,000367454d,1,input/train/0/0/0/000367454d.npy,0,0.346680,0.336182
6,00054fd779,1,input/train/0/0/0/00054fd779.npy,0,0.937012,0.920898
7,0006166539,1,input/train/0/0/0/0006166539.npy,0,1.000000,1.000000
8,00061fc157,0,input/train/0/0/0/00061fc157.npy,0,0.418701,0.432373
9,000698406f,0,input/train/0/0/0/000698406f.npy,0,0.188477,0.169189


In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
X = oof_df[['preds_b0', 'preds_b1']] #, 'preds_b2', 'preds_b7']]
y = oof_df.target.values

In [14]:
X.values.shape

(560000, 2)

In [15]:
reg = LinearRegression().fit(X, y)

In [16]:
reg.coef_

array([0.33756143, 0.65041196], dtype=float32)

In [17]:
reg.intercept_

0.022341937

In [18]:
df_test_b0 = pd.read_csv(f"../final_models/v_5/b0/submission.csv")
df_test_b1 = pd.read_csv(f"../final_models/v_5/b1/submission.csv")
# df_test_b2 = pd.read_csv(f"../final_models/v_4/b2_3d_v4/submission.csv")
# df_test_b7 = pd.read_csv(f"../final_models/v_4/b7_3d_v4/submission.csv")

In [19]:
test_X = np.vstack([df_test_b0.target.values, df_test_b1.target.values]).T #, df_test_b2.target.values, df_test_b7.target.values

In [20]:
sub = pd.read_csv("../input/sample_submission.csv")

In [21]:
sub['target'] = reg.predict(test_X)

In [22]:
sub

,id,target
0,00005bced6,1.010315
1,0000806717,0.956172
2,0000ef4fe1,0.357198
3,00020de251,0.950453
4,00024887b5,0.160246
...,...,...
225995,ffff4125f1,0.280867
225996,ffff9d32a6,0.330954
225997,ffff9f4c1f,0.278093
225998,ffffa19693,1.010315


In [23]:
sub.to_csv("../final_models/v_5/submission.csv", index=False)

In [24]:
df_test_b0 = pd.read_csv(f"../final_models/v_5/b0/submission.csv")
df_test_b1 = pd.read_csv(f"../final_models/v_5/b1/submission.csv")

sub['target'] = df_test_b0.target.values + df_test_b1.target.values

In [26]:
sub.to_csv("../final_models/v_5/submission.csv", index=False)